In [ ]:
df[new_col] = df.apply(lambda x: True if x[col_name] else False, axis = 1) 

# Qiita Fecal Metadata Normalization 
#### by Rebecca Hu
#### 02/21/2019
rows:  40763 cols:  2570
### Added column headers include:
qiita_host_age - normalized age in years
<br> qiita_host_age_units - 'years' repeated in all rows
<br> qiita_host_sex - normalized sex as 'male' or 'female'
<br> qiita_host_ethnicity_white - True if the host identifies as white, False or Nan otherwise
<br> qiita_host_ethnicity_black_or_african_american - True if the host identifies as black or African American, False or Nan otherwise
<br> qiita_host_ethnicity_hispanic_or_latino - True if the host identifies as hispanic or latino, False or Nan otherwise
<br> qiita_host_ethnicity_asian - True if the host identifies as asian, False or Nan otherwise
<br> qiita_host_ethnicity_american_indian_or_alaska_native - True if the host identifies as Native American, False or Nan otherwise
<br> qiita_host_ethnicity_native_hawaiian_or_other_pacific_islander - True if the host identifies as native Hawaiian or Pacific Islander, False or Nan otherwise
<br> qiita_host_ethnicity_other - True if host specified 'other' as ethnicity/race, False or Nan otherwise
<br> qiita_host_ethnicity_multiracial - True if host specified 'multi' or some variation as ethnicity/race, False or Nan otherwise
<br> qiita_host_multiracial - True if host specified multiple ethinicies/races, False or Nan otherwise
<br> qiita_host_ethnicity_combined - string values, most specific host race/ethinicity e.g. 'Japanese-Caucasian'
<br> qiita_host_weight - normalized & cleaned weight in kg
<br> qiita_host_weight_units - 'kg' repeated in all rows
<br> qiita_host_height - normalized and cleaned weight in cm
<br> qiita_host_height_units - 'cm' repeated in all rows
<br> qiita_host_bmi - float values, normalized host bmi
<br> qiita_host_healthy_weight - True if host falls under healthy weight category, False or Nan otherwise
<br> qiita_host_food_allergy - True if host has food allergy 
<br> qiita_host_allergy - True if host has any allergies in allergy dictionaries, False or Nan otherwise
<br> qiita_host_cancer - True if host has any cancer in cancer dictionary, False or Nan otherwise
<br> qiita_host_ibd - True if host has inflammatory bowel disease, False or Nan otherwise
<br> qiita_host_ibd_type - 'cd' if host has Crohn's Disease, 'uc' if host has ulcerative colitis, 'not specified' if host has ibd but does not specify what kind, 'not applicable' if host does not have ibd
<br> qiita_host_diabetes - True if host has diabetes, False otherwise
<br> qiita_host_diabetes_subtype - 'type1' if host has Type I diabetes, 'type2' is host has Type II diabetes, 'not specified' if host has diabetes but does not specify what kind, 'not applicable' if host does not have diabetes
<br> qiita_host_disease - True if host has a miscellaneous disease in disease dictionary, False otherwise
<br> qiita_host_medication - True if host reported taking any medications, False otherwise
<br> qiita_host_healthy - False if host has any disease, allergy, or uses medication, True otherwise
#### Static Columns
qiita_sample_type - 'stool' repeated in all rows
<br> qiita_empo_1 - 'host-associated' repeated in all rows
<br> qiita_empo_2 - 'animal' repeated in all rows
<br> qiita_empo_3 - 'animal distal gut' repeated in all rows
<br> qiita_host_scientific_name - 'Homo sapiens' repeated in all rows
<br> qiita_host_taxid - 9606 repeated in all rows
<br> qiita_host_common_name - 'human' repeated in all rows
<br> qiita_env_feature - 'human-associated habitat' repeated in all rows

In [7]:
import pandas as pd
import numpy as np
import math
import timeit

In [8]:
def contains(string):
    '''
    a function to check if there are any columns names in df that contain "string"
    
    param: any string that you are looking for within the column header
    
    return: a list of column headers in df that contain the inputter string
    '''
    return df.columns[df.columns.str.contains(str(string))]

# Import CSV

In [16]:
def clean_csv(file):
    '''
    reads in the csv and normalizes all variations of true and false values to bool and all variations of null values to nan
    
    param: the filename of the csv as a string
    return: cleaned pandas DataFrame
    '''
    df = pd.read_csv(file,header=0, sep ='\t', decimal = ',',
            true_values =['true','yes','y','Yes','Y','YES', 'Self-diagnosed', 'Diagnosed by a medical professional (doctor, physician assistant)'],
            false_values=['false','no','n','No','N','NO', 'I do not have this condition', 'I do not have this condition'],
            na_values=['Unknown','Unspecified','no_data','not applicable', 'Not applicable' ,'Missing: not collected', 'Missing: not provided',
                      'Missing: Not recorded', 'Missing: Restricted access' , 'Missing', 'Not provided', 'unspecified', 'MISSING: RESTRICTED ACCESS',
                      'not collected', 'not provided', 'missing: not provided', 'MIssing: Not provided', 'Missing: Not reported'],
            low_memory= False
            )
    return df

In [17]:
#Read Fecal Metadata file
df = clean_csv('20190223_all_human_fecal_metadata.txt')
num_row = df.shape[0]
num_col = df.shape[1]
print('rows: ', num_row, 'cols: ', num_col)

rows:  40763 cols:  2570


In [11]:
df.head()

,#SampleID,aborh,achilles_tendinitis,acid_reflux,acne,acne_medication,acne_medication_otc,active_disease,activity_target,acute_kidney_failure,...,years_known_subject,years_smoker,yellow_rice_wine,ygrt,ygrt_freq,yogurt,yogurt_brand,yogurt_live_culture,zipcode,zygosity
0,11669.St.chronic.GLD019.T.60388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10352.MIC20100111.57838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11405.subject631.timepoint0.60671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10317.000022608.56754,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11358.2353.61922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
#display all column headers in the DataFrame

#pd.options.display.max_seq_items = 3000
#df.columns

# Normalize Age

In [6]:
#optmized (using iterows) function to normalize ages and compile them into a list
def normalize_age(df):
    '''
    normalize age columns into years and compile them into a Series
    
    param: DataFrame containing all different age columns
    return: a Series containing all normalized age values
    '''
    age_list = []
    for index, row in df.iterrows():
        
        if not math.isnan(float(row['age'])):
            
            if row['age_unit'] == 'years' or  row['age_units'] == 'years':
                age_list.append(round(float(row['age']), 3))
                
            elif row['age_unit'] == 'weeks' or  row['age_units'] == 'weeks':
                normalized_age = float(row['age']) / 52
                age_list.append(round(normalized_age, 3))
                
            elif row['age_unit'] == 'days' or  row['age_units'] == 'days':
                normalized_age = float(row['age']) / 365
                age_list.append(round(normalized_age, 3))
                
            elif row['age_unit'] == 'minutes' or  row['age_units'] == 'minutes':
                normalized_age = float(row['age']) / 525600
                age_list.append(round(normalized_age, 3))
                
            else:
                age_list.append(float('Nan'))
                
        elif not math.isnan(float(row['age_baby_days'])):
            normalized_age = float(row['age_baby_days']) / 365
            age_list.append(round(normalized_age, 3))
            
        elif not math.isnan(float(row['age_corrected'])):
            age_list.append(round(float(row['age_corrected']), 3)) 
            
        elif not math.isnan(float(row['age_in_years'])):
            age_list.append(round(float(row['age_in_years']), 3))
        
        elif not math.isnan(float(row['age_years'])):
            age_list.append(round(float(row['age_years']), 3))
            
        elif not math.isnan(float(row['host_age'])):
            age_list.append(round(float(row['host_age']), 3))            
        
        else:
            age_list.append(float('Nan'))
            
    return pd.Series(age_list)

In [16]:
#qiita_host_age column 
df['qiita_host_age']= normalize_age(df)

11.4 s ± 207 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# %% result
# 11.4 s ± 207 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [11]:
#qiita_host_age_units column ('years' in all rows)
age_units = ['years']
age_units_list = [] 

for i in range(0, num_row):
    age_units_list += age_units
    
df['qiita_host_age_units'] = pd.Series(age_units_list)

# Normalize Sex

In [21]:
def normalize_sex(df):
    '''
    normalizes sex and gender into 'male' or 'female'
    
    param: DataFrame containing all sex/gender columns
    return: pd.Series containing all normalized sex 
    '''
    sex_list = []
    for index, row in df.iterrows():
        if row['gender'] == 'male' or row['gender'] == 'female':
            sex_list.append(row['gender'])
        elif row['sex'] == 'male' or row['sex'] == 'female':
            sex_list.append(row['sex'])
        else:
            sex_list.append(float('Nan'))
    return pd.Series(sex_list)

In [13]:
#qiita_host_sex column
df['qiita_host_sex'] = normalize_sex(df)

In [22]:
# %%timeit result
# 11.3 s ± 466 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


11.3 s ± 466 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Normalize Ethnicity

In [14]:
#This dict was created by searching through all known ethnicity/race columns and compiling all variations of responses

ethnicity_dict = {
    'white': ['white', 'White' ,'Caucasian', 'caucasian', 'Cauc', 'cauc'],
    'black or african american': ['African', 'african', 'Black-African American', 'Afr Am', 'af am'], 
    'hispanic or latino': ['Hispanic', 'hispanic', 'Latino', 'latino', 'Mestizo', 'Yanomami'], 
    'asian': ['Asian', 'asian', 'japanese'], 
    'american indian or alaska native': ['Amerindian', 'nat am', 'nat Aam', 'native.american'], 
    'native hawaiian or other pacific islander': ['pacific.islander', 'pacific islander', 'Pacific', 'hawaiian'],
    'other': ['other', 'Other'],
    'multi': ['multi', 'Multi', 'more', 'More']
}    

In [23]:
def normalize_ethnicity(df, ethnicity):
    '''
    normalizes a particular ethnicity based on ethnicity dict above
    
    param: 
    - a DataFrame containing all ethnicity/race data
    - a string ethnicity to normalize ('ethnicity' must be a key in the above ethnicity dict)
    
    return: pd.Series containing all normalized, boolean values for the given ethnicity
    
    '''
    ethnicity_list = []
    for index, row in df.iterrows():
        if type(row['race']) == str:
            
            # fix to replace caucasian with white
            curr = row['race']
            if curr == 'Caucasian' or curr == 'caucasian':
                curr = 'white' 
            elif curr == 'Not hispanic or latino':
                curr = 'other'

            for value in ethnicity_dict[ethnicity]:
                if value in curr:
                    ethnicity_list.append(True)
                    break
                elif value == ethnicity_dict[ethnicity][-1]:
                    ethnicity_list.append(False)
                    
        elif type(row['raceethnicity']) == str:
                        
            # fix to replace caucasian with white
            curr = row['raceethnicity']
            if curr == 'Caucasian' or curr == 'caucasian':
                curr = 'white' 
            elif curr == 'Not hispanic or latino':
                curr = 'other'
                
            for value in ethnicity_dict[ethnicity]:
                if value in curr:
                    ethnicity_list.append(True)
                    break
                elif value == ethnicity_dict[ethnicity][-1]:
                    ethnicity_list.append(False)
                    
        elif type(row['ethnicity']) == str:
                        
            # fix to replace caucasian with white
            curr = row['ethnicity']
            if curr == 'Caucasian' or curr == 'caucasian':
                curr = 'white' 
            elif curr == 'Not hispanic or latino':
                curr = 'other'
                
            for value in ethnicity_dict[ethnicity]:
                if value in curr:
                    ethnicity_list.append(True)
                    break
                elif value == ethnicity_dict[ethnicity][-1]:
                    ethnicity_list.append(False)
                    
        elif type(row['ethnic_group']) == str:
                        
            # fix to replace caucasian with white
            curr = row['ethnic_group']
            if curr == 'Caucasian' or curr == 'caucasian':
                curr = 'white' 
            elif curr == 'Not hispanic or latino':
                curr = 'other'
                
            for value in ethnicity_dict[ethnicity]:
                if value in curr:
                    ethnicity_list.append(True)
                    break
                elif value == ethnicity_dict[ethnicity][-1]:
                    ethnicity_list.append(False)
                    
        elif type(row['ethnicgroup']) == str:
                        
            # fix to replace caucasian with white
            curr = row['ethnicgroup']
            if curr == 'Caucasian' or curr == 'caucasian':
                curr = 'white' 
            elif curr == 'Not hispanic or latino':
                curr = 'other'
                
            for value in ethnicity_dict[ethnicity]:
                if value in curr:
                    ethnicity_list.append(True)
                    break
                elif value == ethnicity_dict[ethnicity][-1]:
                    ethnicity_list.append(False)
                    
        elif type(row['racescrq_self_rpt']) == str:
                        
            # fix to replace caucasian with white
            curr = row['racescrq_self_rpt']
            if curr == 'Caucasian' or curr == 'caucasian':
                curr = 'white' 
            elif curr == 'Not hispanic or latino':
                curr = 'other'
                
            for value in ethnicity_dict[ethnicity]:
                if value in curr:
                    ethnicity_list.append(True)
                    break
                elif value == ethnicity_dict[ethnicity][-1]:
                    ethnicity_list.append(False)
                    
        elif type(row['ethnicity_scrq_self_rpt']) == str:
                        
            # fix to replace caucasian with white
            curr = row['ethnicity_scrq_self_rpt']
            if curr == 'Caucasian' or curr == 'caucasian':
                curr = 'white' 
            elif curr == 'Not hispanic or latino':
                curr = 'other'
                
            for value in ethnicity_dict[ethnicity]:
                if value in curr:
                    ethnicity_list.append(True)
                    break
                elif value == ethnicity_dict[ethnicity][-1]:
                    ethnicity_list.append(False)

        else:
            ethnicity_list.append(False)
    return pd.Series(ethnicity_list)

In [16]:
#qiita_host_ethnicity columns 
df['qiita_host_ethnicity_white'] = normalize_ethnicity(df, 'white')
df['qiita_host_ethnicity_black_or_african_american'] = normalize_ethnicity(df, 'black or african american')
df['qiita_host_ethnicity_hispanic_or_latino'] = normalize_ethnicity(df, 'hispanic or latino')
df['qiita_host_ethnicity_asian'] = normalize_ethnicity(df, 'asian')
df['qiita_host_ethnicity_american_indian_or_alaska_native'] = normalize_ethnicity(df, 'american indian or alaska native')
df['qiita_host_ethnicity_native_hawaiian_or_other_pacific_islander'] = normalize_ethnicity(df, 'native hawaiian or other pacific islander')
df['qiita_host_ethnicity_other'] = normalize_ethnicity(df, 'other')
df['qiita_host_ethnicity_multiracial'] = normalize_ethnicity(df, 'multi')

In [17]:
#function to determine multiracial 
def multiracial(df):
    '''
    looks at previously create qiita_host_ethnicity_[blank] columns to determine whether the host has reported multiple ethnicities/races
    
    param: DataFrame containing all race/ethnicity data
    return: pd.Series containing boolean values of wether the host has multiple races/ethnicities reported
    '''
    ethnicity_columns = ['qiita_host_ethnicity_white', 'qiita_host_ethnicity_black_or_african_american',\
                         'qiita_host_ethnicity_hispanic_or_latino', 'qiita_host_ethnicity_asian',\
                         'qiita_host_ethnicity_american_indian_or_alaska_native',\
                         'qiita_host_ethnicity_native_hawaiian_or_other_pacific_islander', 'qiita_host_ethnicity_other',\
                         'qiita_host_ethnicity_multiracial']
    multiracial_list = []
    for index, row in df.iterrows():
        multi = False
        count = 0
        for ethnicity in ethnicity_columns:
            if row['qiita_host_ethnicity_multiracial'] == True:
                multi = True
                break
            elif row[ethnicity] == True:
                count += 1
        if count >= 2 or multi == True:
            multiracial_list.append(True)
            
        else:
            multiracial_list.append(False)
    return pd.Series(multiracial_list)

In [18]:
#qiita_host_multiracial column
df['qiita_host_multiracial'] = multiracial(df)

# Maybe faster way to do this is apply clean_ethnicity_inputs() to entire column/row?

# Also how to make clean_ethnicity_inputs so its not hardcoded?

In [19]:
#function to clean ethnicity imputs
def clean_ethnicity_inputs(entry):
    '''
    cleans the strings in ethnicity inputs 
    param: a string entry in an ethnicity column
    returns: a normalized string
    '''
    entry = entry.replace('nA', 'n A')
    entry = entry.lower()
    entry = entry.replace('half.', '')
    entry = entry.replace('.', ' ')
    entry = entry.replace('black-', '')
    entry = entry.replace('caucasian', 'white')
    entry = entry.replace('mestizohispano', 'mestizo')   
    entry = entry.replace('amerindian', 'american indian')
    return entry   

#functions to normalize ethnicity 
def combined_race(df):
    '''
    creates a column that has all reported host ethncity/races, cleaned but not normalized at all
    eg.) black, japanese, mestizo
    
    param: dataframe
    returns: a pd.Series with merged, but not currated with all ethnicities/races for the given host
    '''
    ethnicity_list = []
    for index, row in df.iterrows():
        if type(row['race']) == str:
            ethnicity_list.append(clean_ethnicity_inputs(row['race']))
        elif type(row['raceethnicity']) == str:
            ethnicity_list.append(clean_ethnicity_inputs(row['raceethnicity']))
        elif type(row['ethnicity']) == str:
            ethnicity_list.append(clean_ethnicity_inputs(row['ethnicity']))
        elif type(row['ethnic_group']) == str:
            ethnicity_list.append(clean_ethnicity_inputs(row['ethnic_group']))
        elif type(row['ethnicgroup']) == str:
            ethnicity_list.append(clean_ethnicity_inputs(row['ethnicgroup']))
        else:
            ethnicity_list.append(float('Nan'))
    return pd.Series(ethnicity_list)

In [20]:
#qiita_host_ethnicity column
df['qiita_host_ethnicity_combined'] = combined_race(df)

## Normalize Weight (in kg)

In [21]:
#function to correct weight in kg
# Everyone should fall between 0.5kg and 200kg, adults should be at least 20kg
def correct_weight(row_index, weight):
    '''
    cleans & normalizes weight inputs with specified bounds to kilograms
    
    param: the row of the host and the weight input
    return: a cleaned and normalized weight value, or nan if the value is out of bounds
    
    '''
    age = df.iloc[row_index]['qiita_host_age']
    
    if not math.isnan(float(df.iloc[row_index]['weight'])):
        if df.iloc[row_index]['weight_units'] == 'kg':
            return round(float(df.iloc[row_index]['weight']), 3)
        elif df.iloc[row]['weight_units'] == 'pounds':
            return round(float(df.iloc[row_index]['weight']) / 2.205 , 3)
                
    if weight < 0.5 or weight > 200:
        return float('Nan')
    if age >= 18:
        if weight < 20:
            return float('Nan')
    else:
        return round(float(weight), 3)
    return round(float(weight), 3)

#function to normalize weight 0.5kg to 200 kg
def normalize_weight(df):
    '''
    imputes cleaned weight values into a pd.Series
    
    param: DataFrame
    return: pd.Series of merged and cleaned values
    '''
    weight_list = []
    for index, row in df.iterrows():
        if not math.isnan(float(row['weight'])):
            weight = correct_weight(index, float(row['weight']))
            weight_list.append(weight)
        elif not math.isnan(float(row['weight_kg'])):
            weight = correct_weight(index, float(row['weight_kg']))
            weight_list.append(weight)
        elif not math.isnan(float(row['tot_mass'])):
            weight = correct_weight(index, float(row['tot_mass']))
            weight_list.append(weight)
        else:
            weight_list.append(float('Nan'))
    return pd.Series(weight_list)

In [22]:
#qiita_host_weight column
df['qiita_host_weight'] = normalize_weight(df)

In [23]:
#qiita_host_weight_units column
weight_units = ['kg']
weight_units_list = []

for i in range(0, num_row):
    weight_units_list += weight_units
    
df['qiita_host_weight_units'] = pd.Series(weight_units_list)

## Normalize Height (in cm)

In [24]:
#function to correct height in cm
# Everyone should fall between 48cm and 210cm, adults should be at least 105cm
def correct_height(row_index, height):
    '''
    cleans & normalizes height inputs with specified bounds to centimeters
    
    param: the row of the host and the height input
    return: a cleaned and normalized height value, or nan if the value is out of bounds
    
    '''
    age = df.iloc[(row_index)]['qiita_host_age']
    if height < 48 or height > 210:
        return float('Nan')
    if age >= 18:
        if height < 20:
            return float('Nan')
    else:
        return round(float(height), 3)
    return round(float(height), 3)

#function to normalize height
def normalize_height(df):
    '''
    imputes cleaned height values into a pd.Series
    
    param: DataFrame
    return: pd.Series of merged and cleaned values
    '''
    height_list = []
    for index, row in df.iterrows():
        if type(row['height']) == str:
            height = row['height'].replace(' cm', '')
            if not math.isnan(float(height)):
                height = correct_height(index, float(height))
                height_list.append(height)
        elif not math.isnan(float(row['height_cm'])):
            height = correct_height(index, float(row['height_cm']))
            height_list.append(height)
        elif not math.isnan(float(row['height_m'])):
            height = correct_height(index, float(row['height_m']) * 100)
            height_list.append(height)
        elif not math.isnan(float(row['height_or_length'])):
            height = correct_height(index, float(row['height_or_length']) * 100)
            height_list.append(height)
        else:
            height_list.append(float('Nan'))
    return pd.Series(height_list)

In [25]:
#qiita_host_height column
df['qiita_host_height'] = normalize_height(df)
#df['qiita_host_height'].value_counts()

In [26]:
#qiita_host_height_units column
height_units = ['cm']
height_units_list = []

for i in range(0, num_row):
    height_units_list += height_units
    
df['qiita_host_height_units'] = pd.Series(height_units_list)

## Normalize BMI

In [27]:
#function to normalize and calculate bmi
#organizes bmi values and calculates it for samples with age > 18 w/o bmi
#some samples only have bmi categories
#corrects by making sure bmi is only between 12 & 42

def correct_bmi(bmi):
    '''
    keeps values within specified BMI bounds
    
    param: a bmi value
    return: a cleaned bmi value, or nan if value is not within bounds
    '''
    if bmi <= 42 and bmi >= 12:
        return round(bmi, 3)
    else:
        return float('Nan')

def normalize_bmi(df):
    '''
    creates a column of merged and cleaned BMI values
    
    param: dataframe
    returns a series with merged and cleaned BMI values
    '''
    bmi_list = []
    for index, row in df.iterrows():
        if not math.isnan(float(row['body_mass_index'])):
            curr = correct_bmi(float(row['body_mass_index']))
            bmi_list.append(curr)
        elif not math.isnan(float(row['bmi_corrected'])):
            curr = correct_bmi(float(row['bmi_corrected']))
            bmi_list.append(curr)
        elif not math.isnan(float(row['bmi'])):
            curr = correct_bmi(float(row['bmi']))
            bmi_list.append(curr)
        #check if has age
        elif not math.isnan(float(row['age'])):
            #check if >18
            if row['qiita_host_age'] >= 18:
                #check if has height and weight
                if not math.isnan(float(row['qiita_host_height'])) \
                and not math.isnan(float(row['qiita_host_weight'])):
                    height = float(row['qiita_host_height']) / 100 # in meters
                    weight = float(row['qiita_host_weight']) # in kg
                    bmi = correct_bmi(weight / ((height) ** 2))
                    bmi_list.append(round(bmi, 3))
                else:
                    bmi_list.append(float('Nan'))
            else:
                bmi_list.append(float('Nan'))
        else:
            bmi_list.append(float('Nan'))
    return pd.Series(bmi_list)

In [28]:
#qiita_host_bmi
df['qiita_host_bmi'] = normalize_bmi(df)

## Normalize Healthy Weight

In [29]:
#function to determine if weight is healthy
#add to only check for bmi if age is >=18
def healthy_weight(df):
    '''
    creates boolean values for hosts of healthy weight
    
    param: dataframe
    returns: a column with boolean values, True if host falls within healthy bmi bounds, false otherwise
    '''
    healthy_weight_list = []
    for index, row in df.iterrows():
        if row['obesity'] == True:
            healthy_weight_list.append(False)
        elif float(row['qiita_host_bmi']) >= 18.5 and float(row['qiita_host_bmi']) <= 24.9:
            healthy_weight_list.append(True)
        elif float(row['qiita_host_bmi']) < 18.5 or float(row['qiita_host_bmi']) > 24.9:
            healthy_weight_list.append(False)
        else:
            healthy_weight_list.append(float('Nan'))
    return pd.Series(healthy_weight_list)

In [30]:
#qiita_host_healthy_weight
df['qiita_host_healthy_weight'] = healthy_weight(df) 

In [31]:
#df['qiita_host_healthy_weight'].value_counts(dropna=False)

## Normalize Allergies

In [32]:
#food allergies only
food_allergy_list = ['peanutallergy', 'shellfishallergy', 'treenutallergy', 'allergic_to_peanuts', 'allergic_to_shellfish',
                     'allergic_to_tree_nuts']

In [33]:
#function for normalizing food allergies
def normalize_food_allergy(df):
    '''
    creates a column normalizing food allergies
    
    param: dataframe
    returns: columns with boolean values, True if any food allergies, false otherwise
    '''
    allergy_list = []
    for index, row in df.iterrows():
        has_allergy = False

        for allergy in food_allergy_list:
            if row[allergy] == False:
                has_allergy = False
            elif row[allergy] == True:
                has_allergy = True
                #if debug: print(allergy + ':' + str(df.iloc[row][allergy]))
                break
                
        if row['allergic_to_i_have_no_food_allergies_that_i_know_of'] == False:
            has_allergy = True

        allergy_list.append(has_allergy)
    return pd.Series(allergy_list)

In [34]:
#qiita_host_food_allergy column
df['qiita_host_food_allergy'] = normalize_food_allergy(df)

In [35]:
#all allergies
bool_allergy_list = ['peanutallergy', 'shellfishallergy', 'treenutallergy', 'allergic_contact_dermatitis', 
        'allergic_rhinitis_', 'allergic_to_other', 
        'allergic_to_peanuts', 'allergic_to_shellfish', 'allergic_to_tree_nuts', 'allergic_to_unspecified', 'beestingallergies', 'drugallergies', 
        'non_food_allergies_beestings', 'non_food_allergies_drug_eg_penicillin', 'non_food_allergies_pet_dander', 
        'non_food_allergies_poison_ivyoak', 'non_food_allergies_sun', 'non_food_allergies_unspecified',  
        'poisonivyoakallergies', 'seasonal_allergies', 'seasonalallergies', 'sunallergies']
str_allergy_list = ['otherallergies']

In [36]:
#function for normalize allergies
def normalize_allergy(df):
    '''
    creates a column normalizing allergies
    
    param: dataframe
    returns: columns with boolean values, True if any allergies, false otherwise
    '''
    allergy_list = []
    for index, row in df.iterrows():
        has_allergy = False

        for allergy in bool_allergy_list:
            if row[allergy] == False:
                has_allergy = False
            elif row[allergy] == True:
                has_allergy = True
                #if debug: print(allergy + ':' + str(df.iloc[row][allergy]))
                break
                
        if row['allergic_to_i_have_no_food_allergies_that_i_know_of'] == False:
            has_allergy = True

        if has_allergy != True:
            for str_allergy in str_allergy_list:
                temp = df[str_allergy].fillna('not provided')
                if temp[index] == 'no' or temp[index] == 'not provided':
                    has_allergy = False
                else:
                    has_allergy = True
                    break
        allergy_list.append(has_allergy)
    return pd.Series(allergy_list)

In [ ]:
#qiita_host_allergy
df['qiita_host_allergy'] = normalize_allergy(df)

## Normalize Cancers

In [ ]:
cancer_list = ['bladder_cancer', 'brain_cancer', 'breast_cancer',
       'cervical_cancer', 'colon_cancer', 'melanoma', 'non_hodgkin_lymphoma',
       'endometrial_cancer', 'kidney_cancer', 'lung_cancer', 'leukemia',
       'non_melanoma_skin_cancer', 'ovarian_cancer', 'pancreatic_cancer',
       'prostate_cancer', 'rectal_cancer', 'stomach_cancer', 'thyroid_cancer']

In [ ]:
#function to normalize cancer
def normalize_cancer(df):
    '''
    creates a column normalizing cancer
    
    param: dataframe
    returns: columns with boolean values, True if any cancers, false otherwise
    '''
    result_list = []
    
    for index, row in df.iterrows():
        has_cancer = False

        for cancer in cancer_list:
            if row[cancer] == True:
                has_cancer = True
                break
        if row['cancer'] == 'Diagnosed by a medical professional (doctor, physician assistant)':
            has_cancer = True
        result_list.append(has_cancer)
        
    return pd.Series(result_list)

In [ ]:
#qiita_host_cancer column
df['qiita_host_cancer'] = normalize_cancer(df)

## Normalize IBD

In [ ]:
#DO NOT INCLUDE 'subset_ibd'
ibd_dict = {'ibd': ['Colitis', 'Yes.IBS', 'Crohns', 'Diagnosed by a medical professional (doctor, physician assistant)'], 
            'ibd_diagnosis': ['Ulcerative colitis', "Crohn's disease"] , 
            'ibd_diagnosis_refined': ['Ulcerative colitis', "Colonic Crohn's Disease", 'Microcolitis'], 
            'gastrointest_disord': ['CD', "Crohn's  Disease", 'UC', "Crohn's Disease", 'IC'],
           'ulcerative_colitis': [True],
            'crohns_disease': [True]
           }

In [ ]:
#function to normalize ibd
def normalize_ibd(df):
    '''
    creates a column normalizing irritable bowel disease
    
    param: dataframe
    returns: columns with boolean values, True if ibd, false otherwise
    '''
    ibd_list = []
    for index, row in df.iterrows():
        has_ibd = False
        for column in ibd_dict:
            temp = df[column].fillna('not provided')
            if temp[index] in ibd_dict[column]:
                has_ibd = True
                break
        ibd_list.append(has_ibd)
    return pd.Series(ibd_list)

In [ ]:
#qiita_host_ibd column
df['qiita_host_ibd'] = normalize_ibd(df)
#df['qiita_host_ibd'].value_counts()

In [ ]:
ibd_true_count = 0
ibd_false_count = 0
for row in range(0, num_row):
    if df.iloc[row]['qiita_host_ibd'] == True:
        if df.iloc[row]['qiita_study_id'] == 10317:
            ibd_true_count += 1
    elif df.iloc[row]['qiita_host_ibd'] == False:
        if df.iloc[row]['qiita_study_id'] == 10317:
            ibd_false_count += 1
#print('true:', ibd_true_count)
#print('false: ', ibd_false_count)

In [ ]:
#qiita_host_ibd_type
def ibd_subtype(df):
    '''
    creates a column specifying ibd subtypes
    
    param: dataframe
    return: a column specifying the specific types of ibd (uc, ic, not specified, or not applicable)
    '''
    subtypes = []
    for index, row in df.iterrows():
        if row['crohns_disease'] == True:
            subtypes.append('cd')
        elif row['ulcerative_colitis'] == True:
            subtypes.append('uc')
        elif row['ibd'] == 'Colitis':
            subtypes.append('uc')
        elif row['ibd'] == 'Crohns':
            subtypes.append('cd')
        elif row['ibd_diagnosis'] == 'Ulcerative colitis':
            subtypes.append('uc')
        elif row['ibd_diagnosis'] == "Crohn's disease":
            subtypes.append('cd')
        elif row['ibd_diagnosis_refined'] == 'Ulcerative colitis':
            subtypes.append('uc')
        elif row['ibd_diagnosis_refined'] == "Colonic Crohn's Disease":
            subtypes.append('cd')
        elif row['gastrointest_disord'] == 'CD' or row['gastrointest_disord'] == 'UC' or row['gastrointest_disord'] == 'IC':
            subtypes.append(row['gastrointest_disord'].lower())
        elif row['gastrointest_disord'] == "Crohn's  Disease" or row['gastrointest_disord'] == "Crohn's Disease":
            subtypes.append('cd')
        elif row['qiita_host_ibd'] == True:
            subtypes.append('not specified')
        else:
            subtypes.append('not applicable')
    return pd.Series(subtypes)

In [ ]:
#qiita_host_ibd_type
df['qiita_host_ibd_type'] = ibd_subtype(df)
#df['qiita_host_ibd_type'].value_counts()

In [ ]:
na_count = 0
ns_count = 0
uc_count = 0
cd_count = 0
for row in range(0, num_row):
    if df.iloc[row]['qiita_host_ibd_type'] == 'not applicable':
        if df.iloc[row]['qiita_study_id'] == 10317:
            na_count += 1
    elif df.iloc[row]['qiita_host_ibd_type'] == 'not specified':
        if df.iloc[row]['qiita_study_id'] == 10317:
            ns_count += 1
    elif df.iloc[row]['qiita_host_ibd_type'] == 'uc':
        if df.iloc[row]['qiita_study_id'] == 10317:
            uc_count += 1
    elif df.iloc[row]['qiita_host_ibd_type'] == 'cd':
        if df.iloc[row]['qiita_study_id'] == 10317:
            cd_count += 1
#print('not applicable: ', na_count)
#print('not specified: ', ns_count)
#print('uc: ', uc_count)
#print('cd: ', cd_count)

## Normalize Diabetes

In [ ]:
#DO NOT INCLUDE 'subset_diabetes'
diabetes_dict = {'diabetes' : ['yes.type.I', 'Diagnosed by a medical professional (doctor, physician assistant)', 'true'], 
                 'diabetes_mellitustype_1' : [True], 
                 'diabetes_mellitustype_2' : [True],
                 'diabetes_type' : ['Type II diabetes']}

In [ ]:
#function to normalize diabetes
def normalize_diabetes(df):
    '''
    creates a columns with boolean values for diabetes
    
    param: dataframe
    return: column wtih boolean values, true if host has diabetes, false otherwise
    '''
    diabetes_list = []
    for index, row in df.iterrows():
        has_diabetes = False
        for column in diabetes_dict:
            temp = df[column].fillna('not provided')
            if temp[index] in diabetes_dict[column]:
                has_diabetes = True
                break
        diabetes_list.append(has_diabetes)
    return pd.Series(diabetes_list)

In [ ]:
#qiita_host_diabetes column
df['qiita_host_diabetes'] = normalize_diabetes(df)
df['qiita_host_diabetes'].value_counts()

In [ ]:
true_count = 0
false_count = 0
for row in range(0, num_row):
    if df.iloc[row]['qiita_host_diabetes'] == True and df.iloc[row]['qiita_study_id'] == 10317:
        true_count += 1
    elif df.iloc[row]['qiita_host_diabetes'] == False and df.iloc[row]['qiita_study_id'] == 10317:
        false_count += 1
print('true:', true_count)
print('false: ', false_count)

In [ ]:
#qiita_host_diabetes_subtype
def diabetes_subtype(df):
    '''
    creates a column specifying types of diabetes
    
    param: dataframe
    returns: a column with specific diabetes types (type1, type2, no type, not applicable)
    '''
    subtypes = []
    for index, row in df.iterrows():
        if row['diabetes_mellitustype_1'] == True:
            subtypes.append('type1')
        elif row['diabetes_mellitustype_2'] == True:
            subtypes.append('type2')
        elif row['diabetes'] == 'yes.type.I':
            subtypes.append('type1')
        elif row['diabetes_type'] == 'Type II diabetes':
            subtypes.append('type2')
        elif row['qiita_host_diabetes'] == True:
            subtypes.append('no type')
        else:
            subtypes.append('not applicable')
    return pd.Series(subtypes)

In [ ]:
#qiita_host_diabetes_subtype
df['qiita_host_diabetes_subtype'] = diabetes_subtype(df)
#df['qiita_host_diabetes_subtype'].value_counts()

In [ ]:
na_count = 0
ns_count = 0
count1 = 0
count2 = 0
for row in range(0, num_row):
    if df.iloc[row]['qiita_host_diabetes_subtype'] == 'not applicable':
        if df.iloc[row]['qiita_study_id'] == 10317:
            na_count += 1
    elif df.iloc[row]['qiita_host_diabetes_subtype'] == 'no type':
        if df.iloc[row]['qiita_study_id'] == 10317:
            ns_count += 1
    elif df.iloc[row]['qiita_host_diabetes_subtype'] == 'type1':
        if df.iloc[row]['qiita_study_id'] == 10317:
            count1 += 1
    elif df.iloc[row]['qiita_host_diabetes_subtype'] == 'type2':
        if df.iloc[row]['qiita_study_id'] == 10317:
            count2 += 1


In [ ]:
#print('not applicable: ', na_count)
#print('not specified: ', ns_count)
#print('type1: ', count1)
#print('type2: ', count2)

## Normalize Miscellaneous Diseases

In [ ]:
true_values =[True, 'True','true','yes','y','Yes','Y','YES', 'Diagnosed by a medical professional (doctor, physician assistant)']
false_values = [False, 'False','false', 'no', 'n', 'No', 'N', 'NO']

In [ ]:
disease_dict = {'cardiovascular_disease': [True], 'celiac_disease': [True],
       'chronicliverdisease_cirrhosis': [True], 'chronicobstructivepulm_disease': [True],
       'disease': ['sinusproblems_candida', 'asthma_allergictopepper', 'sinusproblems', 'asthma', 'asthma_dermatitis',
                    'thyroidhealthproblems', 'allergictopenicillin_fibromyalgia'], 
        'disease_stat': ['hypertension', 'allercic asthma', 'acid reflux', 'gastric heartburn_episodic',
                        'hypertension_acid reflux', 'acid reflux_episodic', 'hypertriglyceridemia',
                        'hypertension_dyslipidemia', 'diabetes', 'cholesterol', 'gastritis',
                         'ischemic cardiopathy_metabolic syndrome', 'liver steatosis_hypertransaminasemia',
                            'hypertransaminasemia'],
       'fibrocystic_breast_disease': [True], 'gastroesophagealreflux_disease': [True],
       'graves_disease': [True], 'hirschsprungs_disease': [True], 'huntingtons_disease': [True],
       'kawasaki_disease': [True], 'kidney_disease': [True], 'liver_disease': [True], 'lung_disease': [True],
       'menieres_disease': [True], 'nonalcoholicfattyliver_disease': [True],
       'osgood_schlatter_disease': [True], 'parkinsons_disease': [True], 'peyronies_disease': [True],
       'polycystic_kidney_disease': [True],
       'von_willebrand_disease': [True], 'acne': true_values, 'acute_kidney_failure': true_values, 'acute_liver_failure': true_values,
       'age_related_cataract': true_values, 'age_related_hearing_loss': true_values, 'age_related_macular_degen': true_values,
       'alopecia_areata': true_values, 'alzheimers': true_values, 'amyotrophic_lateral_sclerosis_': true_values, 'angina': true_values, 
       'aortic_aneurysm': true_values, 'appendicitis': true_values, 'asd': true_values, 'asthma': true_values, 'atherosclerosis': true_values, 
       'atrial_fibrillation': true_values, 'autoimmune_hemolytic_anemia': true_values, 'barretts_esophagus': true_values, 
       'bartholins_cyst': true_values, 'bells_palsy': true_values, 'cdiff': true_values, 'cdiff_positive_yn': true_values, 'chickenpox': true_values,
       'chronic_bronchitis': true_values, 'chronic_kidney_failure': true_values, 'chronic_recurrent_tonsillitis': true_values,
       'chronic_sinusitis': true_values, 'chronicliverdisease_cirrhosis': true_values, 'chronicobstructivepulm_disease': true_values,
       'colon_polyps': true_values, 'congestive_heart_failure': true_values, 'cystic_fibrosis': true_values, 'dandruff': true_values, 
       'deep_vein_thrombosis': true_values, 'depression_bipolar_schizophrenia': true_values, 'dermatographia': true_values, 
       'dilated_cardiomyopathy':true_values, 'diverticulosis': true_values, 'eczema': true_values, 'emphysema': true_values, 'endometriosis': true_values, 
       'epilepsy': true_values, 'epilepsy_or_seizure_disorder': true_values, 'fibromyalgia': true_values, 'gallstones': true_values, 
       'gbs': [], 'gilbert_syndrome': true_values, 'hemochromatosis': true_values, 'hemorrhoids': true_values, 'hiv': true_values, 
                'ibs': ['Diagnosed by a medical professional (doctor, physician assistant)'],
                'irrit_bowel_syndrome': true_values, 'perianal_disease': true_values,
       'irritable_bowel_syndrome_': true_values, 'mental_illness': true_values, 'mental_illness_type_anorexia_nervosa': true_values, 
       'mental_illness_type_bipolar_disorder': true_values, 'mental_illness_type_bulimia_nervosa': true_values, 
       'mental_illness_type_depression': true_values, 'mental_illness_type_ptsd_posttraumatic_stress_disorder': true_values, 
       'mental_illness_type_schizophrenia': true_values, 'mental_illness_type_substance_abuse': true_values,
       'mental_illness_type_unspecified': true_values, 'long_qt_syndrome': true_values, 'malaria': true_values, 'ovarian_cysts': true_values, 
       'patientdied': true_values, 'polycystic_kidney_disease': true_values, 'polycystic_ovary_syndrome_': true_values, 'sickle_cell_anemia': true_values, 
       'sjogrens_syndrome_': true_values, 'urethral_diverticulum': true_values,'urinary_tract_infection_': true_values, 
                'wolff_parkinson_white_syndrome': true_values, 
               'ss_constipation': true_values, 'p3m_constipation': true_values, 'ss_excess_gas': true_values}

In [ ]:
#function to normalize disease
def normalize_disease(df):
    '''
    creates a boolean column for general diseases
    
    param: dataframe
    return: a column with True if any of the above specified columns are true, false otherwise
    '''
    disease_list = []
    for index, row in df.iterrows():
        has_disease = False
        for column in disease_dict:
            temp = df[column].fillna('not provided')
            if temp[index] in disease_dict[column]:
                has_disease = True
                break
        disease_list.append(has_disease)
    return pd.Series(disease_list)

In [ ]:
#qiita_host_disease column
df['qiita_host_disease'] = normalize_disease(df)

## Normalize Medication Use

In [ ]:
med_dict = {'antibiotic': true_values, 'antibiotic_1': ['7_28_AB_for_tooth_extraction', 'Azithromycin_8_30_-_9_3_10_Bronchitis', 'bactrim_skiunknownbscess_7_16_2010_-_7_20_2010'],
            'antibiotic_disturbance': true_values,
       'antibiotic_history': true_values, 'antibiotics': true_values, 'antibiotics_after_birth': true_values,
       'antibiotics_at_birth': true_values, 'antibiotics_past_6_months': true_values, 'antibiotic_select': ['Year', '6 months',
        'Years', 'Month'], 
            'drug_usage': ['TRUE','Ibuprofen','birth control','Claritin','allergy medication','antacids','Prilosec',
                           'Ortho Tri-cyclen Lo','Tums, Advil','Prevacid','Lamictal, lithium carb',
                           'DayQuil','Zantec, Tums, Zertec','Cephalexin','Propecia','OrthoNovum',
                           'Septra, Zofran, chemotherapy drugs','Aleve, Claritin','Levothyroxine',
                           'Ambien','thyroid hormone','Gaviscon','Nasonex',
                           'baby aspirin, Omerprazole, Isosorbride, Lisinopril, Lovastatin, Plavix, Altenolol, Klosnopin, Neurontin, Requip,Finasteride, Acyclovir, Xanax, Nitroglycercin','Zicam, OrthoTriCyclinLo','Tylenol, Oragel','Nuvaring','Fluocinonide cream (eczema)','Omeprazole','baby aspirin, Lipitor, Diovan HCT','albuterol inhaler','Prilosec, Simuastatin, Diazide, Atenolol, Hydroxyzine, Claritan','Evista, Fosamax, Prilosec','Lisinopril, Simvastatin, Hydrochilorol','Actonel','Klonopin, Prozac, Immipramine, Warfarin, Cozaar, Tenex, Calan',
                           'Benadryl','Advil, Benadryl','colace','Equate, Azor, Hydrocholothiazide, aspirin, Nexium',
                           'baby aspirin, Zantac','Advil','baby aspirin, beta-blockers, statins, ace inhibitors','Advil, Lyrica'], 
       'idantibioticdisturbance': true_values, 'p3m_antibiotics': ['|azithromycin|', '|Ciprofloxacin|', '|amoxicillin|',
    '|doxycycline|', '|Augmentin|' , '|Z-PACK|', '|doxycycline|amoxicillin|', '|cephalexin|', '|standard|', '|Leviquin|',
    '|fluconazole|', '|besivance|', '|Flagyl|Ciprofloxacin|', '|amox tr-k clv|', '|keflex|', '|Biaxin|amoxicillin|', '|Rocephin|', 
    '|Niprofurantoin|Ciprofloxacin|', '|nystatin|', '|Ampicillin|'],
       'sampleantibioticdisturbance': true_values, 'subset_antibiotic_history': false_values, 'acne_medication': true_values, 
            'acne_medication_otc': true_values,
       'other_medications': true_values, 'othermedication': true_values, 'drug_usage': true_values}

In [ ]:
#function to normalize medication use
def normalize_medication(df):
    '''
    creates a boolean column for medication use
    
    param: a dataframe
    return: a column of boolean values, True if host used medication, false otherwise
    '''
    med_list = []
    for index, row in df.iterrows():
        med = False
        for column in med_dict:
            temp = df[column].fillna('not provided')
            if temp[index] in med_dict[column]:
                med = True
                break
        med_list.append(med)
    return pd.Series(med_list)

In [ ]:
#qiita_host_medication column
df['qiita_host_medication'] = normalize_medication(df)

## Normalize Host Health

In [ ]:
disease_columns = ['qiita_host_disease', 'qiita_host_diabetes', 'qiita_host_ibd', 'qiita_host_cancer', 'qiita_host_food_allergy',
                  'qiita_host_healthy_weight', 'qiita_host_medication']

In [ ]:
#function to normalize healthy
def normalize_healthy(df):
    '''
    creates a boolean column for a general "healthy" host
    
    param: a dataframe
    return: a boolean column , true if host is healthy and false otherwise
    '''
    healthy_list = []
    for index, row in df.iterrows():
        healthy = True
        for disease in disease_columns:
            if row[disease] == True:
                healthy = False
                break
        healthy_list.append(healthy)
    return pd.Series(healthy_list)

In [ ]:
#qiita_host_healthy
df['qiita_host_healthy'] = normalize_healthy(df)
df['qiita_host_healthy'].value_counts()

# Body Site and Body Habitat Normalization

Body Site Normalization Process:
    1. Take all columns containing body site info by searching for 'body' and/or 'site'
    2. Take all body sites inputs and clean string values
    3. Apply to all rows
   
Body Habitat Normalization Process:
    1. Take all body habitats by searching through all columns with key words
    2. Take all body habitat inputs and clean string values
    3. Take all body site values and give body habitat if none inputted
    4. Apply to all rows

In [18]:
sample = df.sample(frac=0.25, replace=False)
sample.head()

,#SampleID,aborh,achilles_tendinitis,acid_reflux,acne,acne_medication,acne_medication_otc,active_disease,activity_target,acute_kidney_failure,...,years_known_subject,years_smoker,yellow_rice_wine,ygrt,ygrt_freq,yogurt,yogurt_brand,yogurt_live_culture,zipcode,zygosity
8961,10317.000047627.61479,NaN,NaN,I do not have this condition,NaN,False,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39624,10317.000041666.57180,NaN,NaN,I do not have this condition,NaN,False,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38947,10317.000036420.60418,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31723,10352.MIC20270251.57838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33168,10564.FPEBR7P4.61987,AB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
def normalize_body_site(df):
    '''
    normalize body sites
    
    param: df
    returns: series
    '''
    body_sites = ['bodysite_oralmucosa_forehead_volarright_palmright_footright_vag', 
            'host_body_product', 'host_body_site', 'simple_body_site','common_sample_site', 'hmp_site',
            'superbodysiteoralskinunknownsevaginaanalsaureola', 'title_body_site', 'site_sampled', 'body_site']
    
    def clean_body_site(string):
        #once you know the granularity of the body sites, you can add just more to this function to finish
        string = string.replace('UBERON:', '')
        string = string.replace('ICU-FECAL', 'feces')
        string = string.replace('Stool', 'feces')
        string = string.replace('FECAL', 'feces')
        return string
    
    def find_body_site(row):
        for col in body_sites:
            if not pd.isna(row[col]):
                return clean_body_site(row[col])
        return np.nan
    
    result = pd.Series()
    result = df.apply(find_body_site, axis = 1) 
    return result

In [112]:
df['qiita_host_body_site'] = normalize_body_site(df)

In [113]:
df['qiita_host_body_site'].value_counts()

feces              36870
ileum                641
rectum               309
large intestine      184
colon                 77
cecum                 48
Feces                  8
SKBTI089               1
Name: qiita_host_body_site, dtype: int64

In [102]:
contains('body_habitat')

Index(['body_habitat', 'host_body_habitat', 'qiita_host_body_habitat'], dtype='object')

In [106]:
df.body_habitat.value_counts()

UBERON:feces                      23420
UBERON:gastrointestinal system     1075
intestine                           184
feces                               115
Name: body_habitat, dtype: int64

In [115]:
def normalize_body_habitats(df):
    '''
    normalize body habitats
    
    param: df
    returns: series
    '''
    body_habitats = ['body_habitat', 'host_body_habitat']
    
    def clean_body_habitat(string):
        #once you know the granularity of the body habitats, you can add just more to this function to finish
        string = string.replace('UBERON:', '')
        string = string.replace('gastrointestinal system', 'gi tract')
        string = string.replace('intestine', 'gi tract')
        string = string.lower()
        return string
    
    def find_body_habitat(row):
        for col in body_habitats:
            if not pd.isna(row[col]):
                return clean_body_habitat(row[col])
            elif not pd.isna(row['qiita_host_body_site']):
                if row['qiita_host_body_site'] == 'feces':
                    return 'feces'
                else:
                    return 'gi tract'
        return np.nan
    
    result = df.apply(find_body_habitat, axis = 1) 
    return result

In [116]:
df['qiita_host_body_habitat'] = normalize_body_habitats(df)

In [117]:
df['qiita_host_body_habitat'].value_counts()

feces       36879
gi tract     1259
Name: qiita_host_body_habitat, dtype: int64

In [114]:
df.head()

,#SampleID,aborh,achilles_tendinitis,acid_reflux,acne,acne_medication,acne_medication_otc,active_disease,activity_target,acute_kidney_failure,...,ygrt,ygrt_freq,yogurt,yogurt_brand,yogurt_live_culture,zipcode,zygosity,qiita_host_bosy_site,qiita_host_body_site,qiita_host_body_habitat
0,11669.St.chronic.GLD019.T.60388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,feces,feces
1,10352.MIC20100111.57838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,feces,feces
2,11405.subject631.timepoint0.60671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10317.000022608.56754,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,feces,feces,feces
4,11358.2353.61922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,feces,feces


# Atherosclerosis Normalization 

In [123]:
def atherosclerosis(df):
    def merge_as(row):
        if row['atherosclerosis'] == True:
            return True
        else:
            return False
        
    result = df.apply(merge_as, axis = 1)
    return result        

In [124]:
df['qiita_host_atherosclerosis'] = atherosclerosis(df)

In [125]:
df['qiita_host_atherosclerosis'].value_counts()

False    40591
True       172
Name: qiita_host_atherosclerosis, dtype: int64

# Arthritis Normalization

In [135]:
def arthritis(df):
    types = ['osteoarthritis', 'rheumatoid_arthritis']
    def find_body_site(row):
        for col in types:
            if row[col] == True:
                return True
        return False
    
    result = df.apply(find_body_site, axis = 1) 
    return result

In [136]:
df['qiita_host_arthritis'] = arthritis(df)

In [137]:
df['qiita_host_arthritis'].value_counts()

False    40210
True       553
Name: qiita_host_arthritis, dtype: int64

# Alzheimer's Normalization

In [138]:
contains('lzheimer')

Index(['alzheimers', 'alzheimers_dementia', 'alzheimers_dementia_aao'], dtype='object')

In [142]:
df.alzheimers.value_counts(dropna = False)

NaN                                                                  27443
I do not have this condition                                         13289
Diagnosed by a medical professional (doctor, physician assistant)       22
Self-diagnosed                                                           6
Diagnosed by an alternative medicine practitioner                        3
Name: alzheimers, dtype: int64

In [148]:
def alzheimers(df):
    cols = ['alzheimers', 'alzheimers_dementia']
    def find_body_site(row):
        for col in cols:
            if row[col] == True or row[col] == 'Diagnosed by a medical professional (doctor, physician assistant)':
                return True
        return False
    
    result = df.apply(find_body_site, axis = 1) 
    return result

In [149]:
df['qiita_host_alzheimers'] = alzheimers(df)

In [150]:
df['qiita_host_alzheimers'] = alzheimers(df)

False    40730
True        33
Name: qiita_host_alzheimers, dtype: int64

In [143]:
df.alzheimers_dementia.value_counts(dropna = False)

NaN      40450
False      302
True        11
Name: alzheimers_dementia, dtype: int64

In [144]:
df.alzheimers_dementia_aao.value_counts(dropna = False)

NaN      40754
 65.0        2
 81.0        2
 68.0        1
 73.0        1
 80.0        1
 72.0        1
 64.0        1
Name: alzheimers_dementia_aao, dtype: int64

## Static Columns

In [ ]:
#qiita_sample_type column
sample_type = ['stool']
sample_type_list = []

for i in range(0, num_row):
    sample_type_list += sample_type
    
df['qiita_host_sample_type'] = pd.Series(sample_type_list)

In [ ]:
#qiita_empo_1 column 
empo_1 = ['host_associated']
empo_1_list = []

for i in range(0, num_row):
    empo_1_list += empo_1
    
df['qiita_empo_1'] = pd.Series(empo_1_list)

In [ ]:
#qiita_empo_2 column 
empo_2 = ['animal']
empo_2_list = []

for i in range(0, num_row):
    empo_2_list += empo_2
    
df['qiita_empo_2'] = pd.Series(empo_2_list)

In [ ]:
#qiita_empo_3 column 
empo_3 = ['animal distal gut']
empo_3_list = []

for i in range(0, num_row):
    empo_3_list += empo_3
    
df['qiita_empo_3'] = pd.Series(empo_3_list)

In [ ]:
#qiita_host_scientific_name column 
scientific_name = ['Homo sapiens']
scientific_name_list = []

for i in range(0, num_row):
    scientific_name_list += scientific_name
    
df['qiita_host_scientific_name'] = pd.Series(scientific_name_list)

In [ ]:
#qiita_host_taxid column 
taxid = [9606]
taxid_list = []

for i in range(0, num_row):
    taxid_list += taxid
    
df['qiita_host_taxid'] = pd.Series(taxid_list)

In [ ]:
#qiita_host_common_name column
common_name = ['human']
common_name_list = []

for i in range(0, num_row):
    common_name_list += common_name
    
df['qiita_host_common_name'] = pd.Series(common_name_list)

In [ ]:
#qiita_env_feature column
env_feature = ['human-associated habitat']
env_feature_list = []

for i in range(0, num_row):
    env_feature_list += env_feature
    
df['qiita_env_feature'] = pd.Series(env_feature_list)

## Write to CSV

In [ ]:
df.set_index('#SampleID')

In [ ]:
df.to_csv('fecal_03_08_19.tsv', sep = '\t', na_rep = 'not provided', index = False)
df.to_csv('fecal_03_08_19.csv', sep = ',', na_rep = 'not provided', index = False)